## What's Dependency Injection (DI)
Dependency Injection is a way of giving an object (like a class) the things it needs from the outside, instead of creating them itself.

**Why do we need it?**
- Without DI your code becomes tangled, hard to test, and hard to change.
- With DI your code is cleaner, easier to test, and an easy way to swap implementations

### Without DI
**The problem**
- We are creating the `Engine()` dependency from inside the class. Now what happens if I wanna test with a different engine, say `ElectricEngine`, I'll have to modify the `Car` class.
- Well, you can say that why not pass engine as a parameter for the `__init__` method like `__init__(self, engine: Engine)`? Well, that's DI at it's very simplest level.
- This is acceptable but then as dependencies grow, the code becomes messy. Imagine `Engine` also has another dependency `FuelType` and `Car` has other dependencies like `Sensors`, `GPS`, `ParkingCamera`? See, the code gets ugly with all these chained dependencies because you'll literally have to do something like: `__init__(self, engine: Engine(fuel_type: FuelType), sensors: Sensors, gps: GPS, parking_camera: ParkingCamera)`

In [4]:
# Define Dependencies
class Engine:
    def start(self):
        print("Engine started!")
    
# Create class that needs a dependency
class Car:
    def __init__(self):
        self.engine = Engine()

    def drive(self):
        self.engine.start()
        print(f"Car is moving...")

car = Car()
car.drive()

Engine started!
Car is moving...


### With DI
**Why do we need CarModule at all?**
- The Car class says: “I need an Engine.” But who decides which Engine to give (Engine, ElectricEngine, MockEngine for tests)?
- That’s where the module comes in: It contains `@provider` methods that say:
  - "When someone asks for Engine, here’s how to build one."
  - How it knows which model to use? It simple checks the type of the dependency, `Engine` for our case, and then searches through all module's `@provider` methods are returns the one with a return type that matches the requested dependecy

In [5]:
from injector import inject, Module, singleton, provider, Injector

In [ ]:
# Define Dependencies
class Engine:
    def start(self):
        print("Engine started!")

# Create a class that needs the dependency
class Car:
    @inject
    def __init__(self, engine: Engine):  # <- dependency injected
        self.engine = engine

    def drive(self):
        self.engine.start()
        print("car is moving...")

# Create modules that tells `injector` how to build dependencies
class CarModule(Module):
    @singleton
    @provider
    def provide_engine(self) -> Engine:
        return Engine()
    

injector = Injector(modules=[CarModule()]) # building an injector container, providing a list of modules
car = injector.get(Car)
car.drive()

Engine started!
car is moving...
